In [1]:
%matplotlib notebook

import numpy as np
import pandas as pd
import cv2
import os
import math
import itertools
from time import time
import pickle

In [2]:
data_dir = "../data/markup_data/ad_control"

In [4]:
lines = []
img_lines = []
line = [0, 0, 0, 0]

def on_mouse(event, x, y, flags, params):
    global img_lines
    global line
    t = time()

    if event == cv2.EVENT_MBUTTONDOWN:
        print(f"Line starts at: ({x}, {y})")
        line[0] = x
        line[1] = y

    elif event == cv2.EVENT_MBUTTONUP:
        print(f"Line ends at: ({x}, {y})")
        line[2] = x
        line[3] = y
        img_lines.append(line.copy())
        print(img_lines)

i = 0
for img_path in os.listdir(data_dir):
    img_path = data_dir + "/" + img_path
    img = cv2.imread(img_path)

    cv2.namedWindow('real image')
    cv2.setMouseCallback('real image', on_mouse, 0)
    cv2.imshow('real image', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    #ines[i].resize(img_lines.shape)
    lines.append(img_lines)
    img_lines = []

Line starts at: (697, 439)
Line ends at: (706, 449)
[[697, 439, 706, 449]]
Line starts at: (859, 561)
Line ends at: (862, 572)
[[859, 561, 862, 572]]
Line starts at: (425, 421)
Line ends at: (429, 434)
[[425, 421, 429, 434]]
Line starts at: (256, 348)
Line ends at: (262, 344)
[[256, 348, 262, 344]]
Line starts at: (266, 342)
Line ends at: (272, 338)
[[256, 348, 262, 344], [266, 342, 272, 338]]
Line starts at: (153, 278)
Line ends at: (164, 275)
[[153, 278, 164, 275]]
Line starts at: (231, 124)
Line ends at: (239, 136)
[[153, 278, 164, 275], [231, 124, 239, 136]]
Line starts at: (414, 227)
Line ends at: (416, 244)
[[153, 278, 164, 275], [231, 124, 239, 136], [414, 227, 416, 244]]
Line starts at: (357, 469)
Line ends at: (362, 465)
[[357, 469, 362, 465]]
Line starts at: (362, 465)
Line ends at: (358, 462)
[[357, 469, 362, 465], [362, 465, 358, 462]]
Line starts at: (364, 455)
Line ends at: (362, 448)
[[357, 469, 362, 465], [362, 465, 358, 462], [364, 455, 362, 448]]
Line starts at: (680,

Line starts at: (303, 319)
Line ends at: (322, 309)
[[303, 319, 322, 309]]
Line starts at: (320, 574)
Line ends at: (328, 579)
[[320, 574, 328, 579]]
Line starts at: (328, 579)
Line ends at: (328, 579)
[[320, 574, 328, 579], [328, 579, 328, 579]]
Line starts at: (328, 579)
Line ends at: (337, 577)
[[320, 574, 328, 579], [328, 579, 328, 579], [328, 579, 337, 577]]
Line starts at: (637, 478)
Line ends at: (636, 470)
[[320, 574, 328, 579], [328, 579, 328, 579], [328, 579, 337, 577], [637, 478, 636, 470]]
Line starts at: (640, 466)
Line ends at: (645, 456)
[[320, 574, 328, 579], [328, 579, 328, 579], [328, 579, 337, 577], [637, 478, 636, 470], [640, 466, 645, 456]]
Line starts at: (442, 342)
Line ends at: (446, 355)
[[320, 574, 328, 579], [328, 579, 328, 579], [328, 579, 337, 577], [637, 478, 636, 470], [640, 466, 645, 456], [442, 342, 446, 355]]
Line starts at: (446, 355)
Line ends at: (444, 359)
[[320, 574, 328, 579], [328, 579, 328, 579], [328, 579, 337, 577], [637, 478, 636, 470], [640